# Examples
## multimethod
Multimethods are a mapping of signatures (tuple of types) to functions.  They maintain an efficient dispatch tree, and cache the called signatures.

In [ ]:
from multimethod import multimethod
import operator

classic_div = multimethod(operator.truediv)
classic_div[int, int] = operator.floordiv
classic_div

In [ ]:
classic_div(3, 2)

In [ ]:
classic_div(3.0, 2)

In [ ]:
classic_div

Multimethods introspect type annotations and use the name to find existing multimethods.

In [ ]:
import itertools
from collections.abc import Iterable, Sequence


@multimethod
def batched(values: Iterable, size):
    it = iter(values)
    return iter(lambda: list(itertools.islice(it, size)), [])


@multimethod
def batched(values: Sequence, size):
    for index in range(0, len(values), size):
        yield values[index : index + size]


list(batched(iter('abcde'), 3))

In [ ]:
list(batched('abcde', 3))

Multimethods also have an explicit `register` method similar to `functools.singledispatch`.

In [ ]:
@multimethod
def window(values, size=2):
    its = itertools.tee(values, size)
    return zip(*(itertools.islice(it, index, None) for index, it in enumerate(its)))


@window.register
def _(values: Sequence, size=2):
    for index in range(len(values) - size + 1):
        yield values[index : index + size]


list(window(iter('abcde')))

In [ ]:
list(window('abcde'))

## parametric
In addition to `issubclass`, multimethods can dispatch on `isinstance` with parametric checks.

In [ ]:
import asyncio
import inspect
import time
from collections.abc import Callable
from concurrent import futures
from multimethod import parametric

Coroutine = parametric(Callable, inspect.iscoroutinefunction)


@multimethod
def wait(timeout, func, *args):
    return futures.ThreadPoolExecutor().submit(func, *args).result(timeout)


@multimethod
async def wait(timeout, func: Coroutine, *args):
    return await asyncio.wait_for(func(*args), timeout)


wait(0.5, time.sleep, 0.01)

In [ ]:
wait(0.5, asyncio.sleep, 0.01)

In [ ]:
from array import array

IntArray = parametric(array, typecode='i')
isinstance(array('i'), IntArray)

In [ ]:
isinstance(array('f'), IntArray)

## typing subscripts
Support for type hints with subscripts.

In [ ]:
import bisect
import random


@multimethod
def samples(weights: dict):
    """Generate weighted random samples using bisection."""
    keys = list(weights)
    totals = list(itertools.accumulate(weights.values()))
    values = [total / totals[-1] for total in totals]
    while True:
        yield keys[bisect.bisect_right(values, random.random())]


@multimethod
def samples(weights: dict[object, int]):
    """Generate weighted random samples more efficiently."""
    keys = list(itertools.chain.from_iterable([key] * weights[key] for key in weights))
    while True:
        yield random.choice(keys)


weights = {'a': 1, 'b': 2, 'c': 3}
next(samples(weights))

In [ ]:
weights = {'a': 1.0, 'b': 2.0, 'c': 3.0}
next(samples(weights))

## async_multimethod
Async multimethods can dispatch both synchronous and asynchronous functions based on the current context. If called within an async context, it will attempt to call an async function; otherwise, it will call a sync function. If no appropriate function is found, a `DispatchError` is raised.

In [5]:
from multimethod import async_multimethod
import asyncio


class Calculator:
    @async_multimethod
    def add(self, a: int, b: int) -> str:
        return f"sync result: {a + b}"

    @add.register
    async def _(self, a: int, b: int) -> str:
        await asyncio.sleep(0.1)  # Simulate async operation
        return f"async result: {a + b}"


calc = Calculator()

In [6]:
# Synchronous context
result_sync = calc.add(2, 3)
print(f"Synchronous result: {result_sync}")

Synchronous result: sync result: 5


In [7]:
# Asynchronous context
async def async_calculator():
    result_async = await calc.add(5, 7)
    print(f"Asynchronous result: {result_async}")


await async_calculator()

Asynchronous result: async result: 12
